## Charlotte Water Quality Data Scraper
ArcGIS Rest Site: https://cltwmaps.ci.charlotte.nc.us/arcgis/rest/services/Public

In [1]:
#imports
import requests, json, os
import pandas as pd

### Functions

In [2]:
def getServices(baseURL):
    '''Returns a list of services at the root level'''
    params = dict(f='json') #format = json
    j = requests.get(baseURL,params).json()
    return j['services']

In [3]:
def getServiceInfo(baseURL,ID):
    '''Returns the MapServer URL, its name, and the max records'''
    params = dict(f='json') #format = json
    j = requests.get(baseURL,params).json()
    services = j['services']
    service = services[ID]
    serviceURL = baseURL[:-6] + service['name'] + "/" + service['type']
    return (serviceURL,service['name'])

In [4]:
def getLayers(serviceURL):
    '''Returns a list of layers contained in the provided map service'''
    j = requests.get(serviceURL+"?f=pjson").json()
    layers = j['layers']
    return layers 

In [5]:
def getTables(serviceURL):
    '''Returns a list of layers contained in the provided map service'''
    j = requests.get(serviceURL+"?f=pjson").json()
    tables = j['tables']
    return tables 

In [6]:
def getRecordCount(serviceURL, layerID):
    '''Returns the number of records in the supplied layer'''
    layerURL = "{}/{}/query".format(serviceURL,layerID)
    params = dict(f='json',
                  where='OBJECTID >= 0',
                  returnCountOnly='true')
    j = requests.get(layerURL,params).json()
    return j['count']

In [7]:
def getFieldList(serviceURL,layerID):
    '''Returns a list of fields for the supplied layer'''
    layerURL = "{}/{}?f=json".format(serviceURL,layerID)            
    j = requests.get(layerURL).json()
    return j['fields']

In [8]:
def tableToCSV(serviceURL,table,outFolder,overwrite=False):
    '''Saves the data in a table to a CSV file'''
    
    #Get table properties
    tableID = table['id']
    tableName = table['name']
    
    #Set the output name, see if it already exists
    outFN = "{}/{}.csv".format(outFolder,tableName)
    if (os.path.exists(outFN)):
        print("   {} already processed".format(tableName),end=", ")
        if overwrite == False:
            print("Skipping...")
            return
        else:
            print("Overwriting...")
    
    #Get the table query URL
    qURL = "{}/{}/query".format(serviceURL,tableID)

    #Get the number of records
    recordCount = getRecordCount(serviceURL,tableID)
    if recordCount == 0:
        print("\n  No records for {}".format(tableName))
        return
    print("\n  Fetching {} records for {}".format(recordCount,tableName),end=":")

    #Query data 1000 records at a time and add them to a dataframe
    offset = 0
    while offset < recordCount:
        #Status
        print(min(recordCount,offset+1000),end=" ")
        #Get the next 1000 records
        qParams = dict(f='pjson',where='OBJECTID>0',resultOffset=offset,outFields='*')
        j = requests.get(qURL,qParams).json()

        #Get attributes of the selected table
        fields = j['fields']
        features = j['features']

        #Create a dataframe from the first record (transposed)
        if offset == 0: #It's the first record in the dataset
            df = pd.DataFrame(features[0]).T
            startIdx = 1  #We need to skip the first record now...
        else:
            startIdx = 0  #Start at the first record

        #Add the remaining records (transposed)
        for f in features[startIdx:]:
            df = df.append(pd.DataFrame(f).T)

        #Get next set 
        offset+=1000

    #Convert index and date fields (if present)
    for f in fields:
        #Set the OID field as the index
        if f['type'] == 'esriFieldTypeOID':
            df.set_index(f['name'],inplace=True)
        #Convert dates
        if f['type'] == 'esriFieldTypeDate':
            name = (f['name'])
            df[name] = pd.to_datetime(df[name],unit='ms')

    #Write to a file
    outFN = "{}/{}.csv".format(outFolder,table['name'])
    print("\n  {} records saved to {}".format(len(df),outFN))
    df.to_csv(outFN)

In [9]:
def layerToCSV(serviceName,layer,outFolder,overwrite=False):
    '''Saves the data in the REST layer to a CSV (Point FCs only)'''
    #Get table properties
    layerID = layer['id']
    layerName = layer['name']
    
    #Set the output name, see if it already exists
    outFN = "{}/{}.csv".format(outFolder,layerName)
    if (os.path.exists(outFN)):
        print("   {} already processed".format(layerName),end=", ")
        if overwrite == False:
            print("Skipping...")
            return
        else:
            print("Overwriting...")
    
    #Get the table query URL
    qURL = "{}/{}/query".format(serviceURL,layerID)
    
    #Skip group layers that point to sub layers
    if layer['subLayerIds'] != None:
        print("\n  {} is a group layer...".format(layerName))
        return 

    #Get the number of records (see if it's < 1000)
    recordCount = getRecordCount(serviceURL,layerID)
    if recordCount == 0:
        print("\n  No records for {}".format(layerName))
        return
    print("\n  Fetching {} records for {}".format(recordCount,layerName),end=":")

    #Loop through 1000 records at a time
    offset = 0
    while offset < recordCount:
        #Get the next 1000 records
        qParams = dict(f='pjson',where='OBJECTID>0',resultOffset=offset,outFields='*',outSR='4326',datumTransformation='1188')
        print(min(recordCount,offset+1000),end=" ")
        
        #Retrieve data as a JSON object
        j = requests.get(qURL,qParams).json()

        #Get attributes of the selected table
        fields = j['fields']
        features = j['features']

        #Create a dataframe from the first record (transposed)
        if offset == 0: #It's the first record in the dataset
            feature = features[0]
            df = pd.DataFrame.from_dict(feature['attributes'],orient='index').T
            #Get and add the coordinates
            if 'geometry' in feature.keys():
                df['x']=feature['geometry']['x']
                df['y']=feature['geometry']['y']
            startIdx = 1
        else:
            startIdx = 0

        #Add the remaining records (transposed)
        for feature in features[startIdx:]:
            df2 = pd.DataFrame.from_dict(feature['attributes'],orient='index').T
            if 'geometry' in feature.keys():
                df2['x']=feature['geometry']['x']
                df2['y']=feature['geometry']['y']
            df = df.append(df2)

        #Get next set 
        offset+=1000

    #Convert fields
    for f in fields:
        #Set the OID field as the index
        if f['type'] == 'esriFieldTypeOID':
            df.set_index(f['name'],inplace=True)
        #Convert dates
        if f['type'] == 'esriFieldTypeDate':
            name = (f['name'])
            df[name] = pd.to_datetime(df[name],unit='ms')

    #Write to a file
    outFN = "{}/{}.csv".format(outFolder,layer['name'])
    print("\n  {} records saved to {}".format(len(df),outFN))
    df.to_csv(outFN)    

### Main
The Charlotte REST server lists a number of Map Services with one or more layer

In [10]:
#Scrape all data (Tables and Layers)
baseURL = 'https://cltwmaps.ci.charlotte.nc.us/arcgis/rest/services/Public'

for serviceIndex in (1,2,3,4,5,6,7,8,9,13):

    #Get the map service and its name
    serviceURL,serviceName = getServiceInfo(baseURL,serviceIndex)
    print("\n-------\nPROCESSING {}".format(serviceName))
    
    #Create a folder to hold the results
    outFolder = '../Data/Charlotte/'+serviceName[7:]
    if not os.path.exists(outFolder): os.mkdir(outFolder)

    #Get a list of tables and layers
    tables = getTables(serviceURL)
    layers = getLayers(serviceURL)
    
    #Loop through tables hosted in the service and save to CSV
    for i, table in enumerate(tables):
        tableToCSV(serviceURL,table,outFolder)
        
    #Loop through the layers hosted in the service and save to CSV
    for i, layer in enumerate(layers):
        layerToCSV(serviceURL,layer,outFolder,overwrite=True)
    
print("Finished!")


-------
PROCESSING Public/CLTW_DrinkingWaterCHLORINE2
   Free Chlorine Results (2015) already processed, Skipping...
   Free Chlorine already processed, Overwriting...

  Fetching 508 records for Free Chlorine:508 
  508 records saved to ../Data/Charlotte/CLTW_DrinkingWaterCHLORINE2/Free Chlorine.csv

-------
PROCESSING Public/CLTW_DrinkingWaterDBP
   Total Trihalomethanes Results (2015) already processed, Skipping...
   Haloacetic Acids Results (2015) already processed, Skipping...
   Total Trihalomethanes already processed, Overwriting...

  Fetching 24 records for Total Trihalomethanes:24 
  24 records saved to ../Data/Charlotte/CLTW_DrinkingWaterDBP/Total Trihalomethanes.csv
   Haloacetic Acids already processed, Overwriting...

  Fetching 24 records for Haloacetic Acids:24 
  24 records saved to ../Data/Charlotte/CLTW_DrinkingWaterDBP/Haloacetic Acids.csv

-------
PROCESSING Public/CLTW_DrinkingWaterINORGANIC2
   Fluoride Results (2015) already processed, Skipping...
   Arsenic R